In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
from shapely.geometry import shape, Point, MultiPoint
from shapely.ops import nearest_points

In [2]:
with open('./results-7-31-2019.json') as f:
    # flatten json structures
    data = json_normalize(json.load(f))
    # Remove non-Chicago entries
    data = data[data['address'].str.contains('Chicago, IL')]

In [3]:
with open('./CTA_RailStations.geojson') as f:
    stations_geo = json.load(f)

In [4]:
with open('./chicago_neighborhoods.geojson') as f:
    neighborhoods_geo = json.load(f)

In [5]:
# Make the neighborhood json easier to work with
neighborhood_polygons = [{'geometry': shape(feature['geometry']), 'properties': feature['properties']} for feature in neighborhoods_geo['features']]

In [6]:
def get_neighborhood(lon, lat):
    hoods = [polygon['properties']['pri_neigh'] for polygon in neighborhood_polygons if polygon['geometry'].contains(Point(lon, lat))]
    return hoods[0] if len(hoods) > 0 else None

# Add neighborhoods to entries
data['neighborhood'] = data \
    .apply(lambda row: get_neighborhood(row['hdpData.homeInfo.longitude'], row['hdpData.homeInfo.latitude']), axis=1)

In [7]:
# Remove anything not in a neighborhood (probably incorrectly labeled as being in Chicago
data = data[data['neighborhood'].notnull()]
data[['address', 'neighborhood']]

,address,neighborhood
192,"12841 S Union Ave, Chicago, IL",West Pullman
193,"12816 S Sangamon St, Chicago, IL",West Pullman
194,"901 W 129th Pl, Chicago, IL",West Pullman
199,"915 W Vermont Ave, Chicago, IL",West Pullman
202,"12907 S Normal Ave, Chicago, IL",West Pullman
203,"442 W 129th Pl, Chicago, IL",West Pullman
207,"12766 S Union Ave, Chicago, IL",West Pullman
209,"1000 W 129th Pl, Chicago, IL",West Pullman
215,"552 W 129th Pl, Chicago, IL",West Pullman
216,"12819 S Sangamon St, Chicago, IL",West Pullman


In [8]:
station_points = [Point(point['geometry']['coordinates'][1], point['geometry']['coordinates'][0]) for point in stations_geo['features']]

In [9]:
# Get nearest CTA stations
p = Point(data.iloc[0]['hdpData.homeInfo.longitude'], data.iloc[0]['hdpData.homeInfo.latitude'])
np = nearest_points(p, MultiPoint(station_points))

In [ ]:
# save cleaned data
data.to_csv('results-7-31-2019-cleaned.csv')